In [1]:
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary
import spacy as sp
from spacy import displacy
import torch
from eda import lem
import pickle
from concurrent.futures import  ProcessPoolExecutor

In [2]:
vec_model= sp.load('en_core_web_lg')
df = pd.read_csv(r'./pw_services.csv').dropna()

parse_list=lambda s:eval(s)
des = df.Description
cats = df.Categories.apply(parse_list)

In [23]:
cat_vec = {}
for tok_ in cats.tolist():
    for tok in tok_:
        if tok in cat_vec:
            continue
        cat_vec[tok] = vec_model(tok)

def getTagSimMatrix(idx):
    sim_d = {}
    doc = vec_model(df.loc[idx,'Description'])
    for tok in cats[idx]:
        tok_sims = {}
        for n_tok in doc.noun_chunks:
            tok_sims[n_tok] = cat_vec[tok].similarity(n_tok)
        sim_d[tok] = tok_sims
    return sim_d
def recurFindVerb(noun_chunk):
    tok = noun_chunk.root
    
    while tok.has_head:
        if tok.dep_ in {'nsubj','prep'} :
            return None
        if tok.head == tok:
            break
        elif tok.head.pos_ == 'VERB':
            tok = tok.head
            return tok
        elif tok.head.pos_ == 'NOUN':
            return None
        tok = tok.head
top_noun_chunks = lambda idx :pd.DataFrame(getTagSimMatrix(idx)).T.mean().sort_values(ascending=False)
chunks = lambda top_noun_chunks_ : [(chunk.text,score) for chunk,score in top_noun_chunks_[:int(0.5*top_noun_chunks_.size)].items()]


In [20]:
pd.DataFrame(getTagSimMatrix(6)).T

<ipython-input-19-56187c21404a>:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  tok_sims[n_tok] = vec_model(tok).similarity(n_tok)


,"(The, 100A, API)",(users),"(any, type)","(textual, information)",(articles),"(white, papers)",(books),"(code, backups)",(JSON),(HTMLmarkup),...,(You),"(your, data)",(anyone),(it),(yourself),"(a, private, password)","(Private, data)",(you),"(a, valid, password, -, key)","(a, data, extraction, &, management, service)"
storage,0.210751,0.307467,0.346697,0.288041,0.183973,0.233925,0.194981,0.427703,0.086529,0.0,...,0.238880,0.524502,0.126712,0.257464,0.177350,0.368312,0.502982,0.238880,0.328890,0.537293
application development,0.330608,0.429583,0.493208,0.451126,0.285954,0.270668,0.229542,0.379326,0.106005,0.0,...,0.339272,0.521420,0.300491,0.407612,0.242861,0.432029,0.503662,0.339272,0.465470,0.642960
artificial intelligence,0.174130,0.198882,0.354140,0.362263,0.211878,0.316755,0.175138,0.220549,-0.035210,0.0,...,0.272686,0.387226,0.292041,0.336505,0.244712,0.232848,0.368561,0.272686,0.252185,0.425650
data,0.296234,0.484222,0.413388,0.614800,0.271294,0.263547,0.179523,0.514592,0.197926,0.0,...,0.292992,0.840925,0.211444,0.305796,0.165029,0.462213,0.828303,0.292992,0.512510,0.740117
documents,0.185314,0.348419,0.377979,0.577973,0.451364,0.444649,0.373173,0.385624,0.087449,0.0,...,0.281128,0.509432,0.254730,0.303681,0.172078,0.388074,0.504743,0.281128,0.428051,0.453355
file sharing,0.290416,0.521668,0.491890,0.453008,0.289265,0.283100,0.272863,0.546307,0.239008,0.0,...,0.434392,0.626639,0.407146,0.443960,0.329298,0.598888,0.609398,0.434392,0.587354,0.566849


In [24]:
df['chunks'] = df.index.to_series().apply(lambda x :chunks(top_noun_chunks(x)))

<ipython-input-23-3dde0650ed32>:14: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  tok_sims[n_tok] = cat_vec[tok].similarity(n_tok)


In [40]:
idx = 196
print(df.iloc[idx,1])
print(df.iloc[idx,-1])

The ADL Experience (xAPI) allows developers to integrate performance tracking and analysis capabilities in applications. The API provides a data sharing interface for applications that track human performance and behavioral trends. It supports the deployment of both human and machine readable analytical tools in the retrieval of big data about human performance trends and related instructional materials. Users can deploy the API to track the behaviors that people exhibit when reading articles, answering quizzes, watching movies, chatting, playing games, and interacting with mobile apps, among others. The xAPI operates as a cross-platform solution for a variety of software applications including mobile devices, wearable gadgets, and education management systems, among others. ADL Initiative is a US government program to promote policies for learning through the use of technology.
[('big data', 0.520617984012277), ('a data sharing interface', 0.48882806301882814), ('education management 

In [9]:
list(filter(lambda x :x[0]!=None,[(recurFindVerb(n_c),n_c) for n_c in doc.noun_chunks]))

[(selling, Bitcoins),
 (gives, members),
 (gives, access),
 (provides, several APIs),
 (return, ticker information),
 (provides, API methods),
 (checking, the Bitcoin price index history),
 (getting, account information),
 (performing, trades)]

In [10]:
displacy.render(doc,style='dep')


In [6]:
from nltk import pos_tag_sents
pos_tags = pos_tag_sents(des.apply(lem))

In [8]:
# nn_sents = pd.Series([' '.join([e[0] for e in li]) for li in pos_tags])
# nn_sents.dropna()

0        rebro kommun rebro municipality sweden provide...
1        csfd api program interface interact czech slov...
2        ternity decentralize blockchain system applica...
3        api provide method source liquidity mesh kyber...
4        fact russian website dedicate help people beco...
                               ...                        
20758    zync upwire api enable developer set event tri...
20759    zyncro enterprise social networking service pr...
20760    zynx health api provide access zynx content fa...
20761    zype api integrate video feature application a...
20762    zypr application api design work across variet...
Length: 20763, dtype: object

In [7]:
nn_words = []
def dup_nn(li):
    res = ''
    nns = set()
    for w,att in li:
        if att.startswith('NN') or att.startswith('JJ'):
            res += w+' '
            nns.add(w)
        # res += w+' '
    nn_words.append(nns)
    return res
nn_sents = pd.Series([dup_nn(sent) for sent in pos_tags])

In [8]:
# nn_docs = nn_sents.apply(lambda x:vec_model(x))
nn_docs = cats.apply(lambda x:' '.join(set(x))).apply(vec_model)
def get_most_sim(sent_doc,nns,topn = 3):
    words = []
    if not sent_doc:
        return []
    if len(sent_doc)<2:
        return []
    # words = [ (w,max([tag.similarity(vec_model.vocab[w]) for tag in sent_doc])) for w in nns]
    for w in nns:
        ve = vec_model.vocab[w]
        sim = sent_doc.similarity(ve)
        words.append((w,sim))
    # words.sort(key= lambda x:x[1])
    return words
nn_en = pd.Series([[(t,)*round(5*wei) for t,wei in get_most_sim(doc,nns)] for doc,nns in zip(nn_docs,nn_words)])

<ipython-input-8-94de49c9adb3>:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = sent_doc.similarity(ve)


1-将内容中名词简单重复
2-将所有名词按和tag的相似度来进行重复n次


In [9]:
nn_en = nn_en.apply(lambda x: ' '.join([' '.join(en) for en in x]))
nn_sents = nn_sents+' '+nn_en
df['des'] = nn_sents


In [10]:
nn_sents[0]

'rebro kommun rebro municipality sweden provide portal government data information waste recycle library transit politics sport culture business tourism much information map restful api data xml json site documentation swedish  documentation documentation culture culture site sport sport government government government sweden sweden api much much map map   waste waste swedish xml  provide provide recycle data data data data transit transit portal portal library library municipality municipality politics politics tourism tourism business business information information information '

In [11]:
nn_vecs = nn_sents.apply(vec_model)

In [12]:
rel_cats = [[(t,d.similarity(vec_model.vocab[t])) for t in tags] for d,tags in zip(nn_vecs,cats)]

<ipython-input-12-3b118e43b4a0>:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  rel_cats = [[(t,d.similarity(vec_model.vocab[t])) for t in tags] for d,tags in zip(nn_vecs,cats)]


In [13]:
df['new_cats']= [set(x[0] for x in list(filter(lambda x:x[1]>0.5,li))) for li in rel_cats]

In [41]:
df.to_csv('processed.csv',index=None)

In [15]:
dddd = pd.read_csv('processed.csv')

In [68]:
dddd

,Name,Description,Categories,new_cats,des
0,Ã–rebro kommun,rebro kommun rebro municipality Sweden ...,"['government', 'data', 'data', 'nordic']","{'government', 'data'}",rebro kommun rebro municipality sweden provide...
1,ÄŒSFD,The CSFD API is a programming interface to int...,"['media', 'search', 'database']","{'search', 'database', 'media'}",csfd api program interface interact czech slov...
2,Æternity REST API v2,ternity is a decentralized blockchain system ...,"['blockchain', 'bitcoin', 'community']",{'community'},ternity blockchain system application educatio...
3,0x,The 0x API provides methods to source liquidit...,"['blockchain', 'cryptocurrency', 'currency', '...","{'currency', 'financial'}",api provide method source liquidity mesh kyber...
4,100 Facts About Me,100 Facts About Me is a Russian website dedica...,"['social', 'russian']",{'social'},fact russian website dedicate help people good...
...,...,...,...,...,...
20758,Zync Upwire,The Zync Upwire API enables developers to set ...,"['messaging', 'campaigns', 'email', 'voice']",{'messaging'},zync upwire api enable developer event trigger...
20759,Zyncro,Zyncro is an enterprise social networking serv...,"['enterprise', 'social', 'file sharing', 'blog...","{'social', 'enterprise'}",zyncro enterprise social networking service fe...
20760,Zynx Health,The Zynx Health API provides access to Zynx Co...,"['healthcare', 'content', 'data', 'developers'...","{'healthcare', 'financial', 'management', 'dat...",zynx health api access zynx content healthcare...
20761,Zype,The Zype API integrates video features into ap...,"['video', 'analytics', 'business', 'platform-a...",{'business'},zype api integrate video feature application a...
